In [ ]:
# import libraries
import numpy as np
import pandas as pd
from pandas import ExcelWriter
import time
from datetime import datetime
import datetime as dt
import warnings
warnings.simplefilter("ignore")

In [ ]:
# output file name and location
current_date = time.strftime("%Y%m%d")
output_file = 'natural_persons_main' + current_date + '.xlsx'

In [ ]:
# import data
file_path = '../data/report_2.xlsx'
df = pd.read_excel(file_path)

***

In [ ]:
# data preprocessing
# select only natural persons
df_natural = df[df['Party Type']=='Natural Person']

In [ ]:
#Preprocessing party name
df_natural.loc[:,'Party Name_pre']=df_natural['Party Name'].copy()
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str \
                                                           .lower() \
                                                           .replace(r'[^\w\s]',"")

In [ ]:
#Prepocessing 'Identification Document Number' remove space & casing
df_natural.loc[:,'Identification Document Number_pre']=df_natural['Identification Document Number'].copy()
df_natural['Identification Document Number_pre'] = df_natural['Identification Document Number_pre'].str \
                                                                                                   .lower() \
                                                                                                   .replace(r'[^\w\s]',"")

In [ ]:
# create beneficiary only list
bene_only = df_natural.groupby('Party ID')['Relationship (Party Role)'] \
                      .agg(set) \
                      .reset_index(name = 'role_set')

bene_only = bene_only[bene_only['role_set'] == {'Beneficiary'}]
bene_only_list = bene_only['Party ID'].to_list()

df_natural['bene_only'] = df_natural['Party ID'].isin(bene_only_list)

In [ ]:
#dates
df_natural['Date of Birth']  = pd.to_datetime(df_natural['Date of Birth'], format='%Y-%m-%d', errors='coerce').dt.date

In [ ]:
# add universal columns
df_natural['Batch'] = input("Year, Quarter: (eg. format: 2023 Q1)")
df_natural[['Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Acceptable Exception']] = ""

***

In [ ]:
# np1: same name & dob but passport/id country != nationality
# location: sheet 1
tab1 = df_natural.copy()

#create list of Party IDs with HK permanent ID
HK_permID = tab1[tab1['Identification Document Type'] == 'HK Permanent ID']
HK_permID = HK_permID.drop_duplicates(subset = ['Party ID'])
HK_permID_list = HK_permID['Party ID'].tolist()

#create list of British Overseas Territories Passports (obtained from google)
Brit_set = {'Anguilla','Bermuda','Virgin Islands','Cayman Islands','Gibraltar','Guernsey','Jersey','Manx','Montserrat',
            'Saint Helena','Turks and Calcos Islands','Isle Of Man'}

# identification Document Type ==  'Passport'
tab1 = tab1[tab1['Identification Document Type'] == 'Passport']

#create group identifier: [key]
tab1['key'] = tab1['Party Name_pre'].astype(str) + \
              tab1['Date of Birth'].astype(str)

#Replace Brit Overseas Territories Passports with UK passports
tab1_Brit = tab1.copy()
tab1_Brit.loc[tab1['Identification Document Issue Country'].isin(Brit_set), 
              'Identification Document Issue Country'] = "United Kingdom"

# create group identifier: [passport key]
tab1['passport_key'] = tab1_Brit['Party Name_pre'].astype(str) + \
                       tab1_Brit['Date of Birth'].astype(str) + \
                       tab1_Brit['Identification Document Issue Country'].astype(str)

# create group identifier: [nationality key]
tab1['nationality_key'] = tab1['Party Name_pre'].astype(str) + \
                          tab1['Date of Birth'].astype(str) + \
                          tab1['Nationality'].astype(str)



#create list of nationalities
tab1_nationality_list = tab1.nationality_key.tolist()
#check if passports can be found in list of nationalities, flag if cannot be found
tab1['flag'] = ~(tab1['passport_key'].isin(tab1_nationality_list))

#create list of passports
tab1_pp_list = tab1.passport_key.tolist()
#check if nationalities can be found in list of passports, flag if cannot be found
tab1['flag2'] = ~(tab1['nationality_key'].isin(tab1_pp_list))

#filter for records where passports cannot be found in nationalities and vice versa
tab1_filter = tab1[(tab1['flag'] == True) | 
                   (tab1['flag2'] == True)]
          
#create list of parties to flag based on "key" [party name + DOB]
tab1_filter_list = tab1_filter.key.tolist()

#retrieve all records that is in list of parties to flag
tab1_columns = tab1[(tab1['key'].isin(tab1_filter_list))]

#indicate if party ID has HK Perm ID, if nationality- HK
tab1_columns['HK Permanent ID?'] = ""
tab1_columns.loc[((tab1_columns['Party ID'].isin(HK_permID_list)) & 
                  (tab1_columns['Nationality'] == "Hong Kong")),
                 'HK Permanent ID?'] = 'True'

# state inconsistency type in new column 'Inconsistency'
tab1_columns['Inconsistency'] = 'Same Name & DOB but Passport ID Doc Ctry <> Nationality'

#drop duplicates
tab1_columns = tab1_columns.drop_duplicates(subset = ['Party ID',
                                                      'Identification Document Type',
                                                      'Identification Document Issue Country',
                                                      'Nationality'])

In [ ]:
# np2: same name & dob but nationality != id proof (SG, Swiss, Italian, Germany, French, Liechtenstein)
# location: sheet 1
tab2 = df_natural.copy()

# create group identifier: [key]
tab2['key'] = tab2['Party Name_pre'].astype(str) + \
              tab2['Date of Birth'].astype(str)

# 2.1 checking for sg pink NRIC
tab2_1 = tab2[tab2['Identification Document Type'] == 'SG Pink NRIC']

# create a [Nationality] set for each group
tab2_nationality = tab2_1.groupby('key')['Nationality'] \
                        .agg(set) \
                        .reset_index(name = 'nationality_set')

nationality_set = tab2_nationality.nationality_set.tolist()

# check if Singapore exist in [Nationality]
def check_singapore(nationality_set): #check if Singapore exists in the set of Nationality
    singapore_set = {'Singapore'}
    result = nationality_set.difference(singapore_set)
    if 'Singapore' in nationality_set: return 'correct'
    else: return result
    
tab2_nationality['check_singapore'] = tab2_nationality['nationality_set'].apply(lambda x: check_singapore(x))
tab2_1_error = tab2_nationality[tab2_nationality['check_singapore'] != 'correct']

# 2.2 checking for national id of eu countries
# select rows  with national id of eu countries
nationality_list = ['Switzerland', 'Italy', 'Germany', 'France', 'Liechtenstein']
tab2_2 = tab2[tab2['Identification Document Type'] == 'National ID']
tab2_2_filter = tab2_2[(tab2_2['Identification Document Issue Country'].isin(nationality_list))]

# create a [Nationality] set for each group
tab2_2_nationality = tab2_2_filter.groupby('key')['Nationality'] \
                                  .agg(set) \
                                  .reset_index(name='nationality_set')

tab2_nationality_set = tab2_2_nationality.nationality_set.tolist()

# create a [Identification Document Issue Country] set for each group
tab2_2_idctry = tab2_2_filter.groupby('key')['Identification Document Issue Country'] \
                             .agg(set) \
                             .reset_index(name='idctry_set')

tab2_2_idctry['nationality_set'] = tab2_nationality_set

# compare two sets: set(idcrty) - set(nationality)
def compare_country_sets_reverse(idctry_set, nationality_set): #compare difference btw {ID doc ctry} and {nationality}
    result = idctry_set.difference(nationality_set)
    if result: return result #if result not empty, return result
    else: return 'correct' #if result IS empty, return 'correct'

# extract groups whose sets is not identical     
tab2_2_idctry['set_diff'] = tab2_2_idctry.apply(lambda x:
                                                compare_country_sets_reverse(x.idctry_set, x.nationality_set), 
                                                axis = 1)
tab2_2_error = tab2_2_idctry[tab2_2_idctry['set_diff'] != 'correct']

# v-lookup in originial df 
tab2_1_error_list = tab2_1_error['key'].tolist()
tab2_2_error_list = tab2_2_error['key'].tolist()
tab2_error_list = tab2_1_error_list + tab2_2_error_list

tab2['exist'] = tab2['key'].isin(tab2_error_list)
tab2_columns = tab2[tab2['exist'] == True]

# state inconsistency type in new column 'Inconsistency'
tab2_columns['Inconsistency'] = \
'Same Name & DOB but Nationality <> ID proof(SG, Swiss, Italian, Germany, French, Liechtenstein)'

#drop duplicates
tab2_columns = tab2_columns.drop_duplicates(subset=['Party ID','Identification Document Type',
                                                    'Identification Document Issue Country','Nationality'])

In [ ]:
# np4: same name & dob but diff place of birth
# location: sheet 1
tab4 = df_natural.copy()

# create concat column of same party name & same DOB
tab4['key'] = tab4['Party Name_pre'].astype(str) + \
              tab4['Date of Birth'].astype(str)

# add a column to count no. of Place of Birth based on same Party Name & DOB
tab4['unique_count'] = tab4.groupby(['key'])['Place of Birth'] \
                           .transform('nunique')

# keep only rows where theres > 1 Place of Birth 
tab4 = tab4[tab4['unique_count'] > 1]

# remove duplicates
tab4_columns = tab4.drop_duplicates(['Party ID','key','Place of Birth'])

# state inconsistency type in new column 'Inconsistency'
tab4_columns['Inconsistency'] = 'Same Name, DOB, Diff Place of Birth'

In [ ]:
# np5: same name & dob & address type but diff address country
# location: sheet 1
tab5 = df_natural.copy()

tab5 = tab5[tab5['Address Type'] == 'Existing Residential Address']

# create concat column of same party name, DOB
tab5['key'] = tab5['Party Name_pre'].astype(str) + tab5['Date of Birth'].astype(str)

# add a column to count no. of Address - Country based on same Party Name & DOB
tab5['unique_count'] = tab5.groupby(['key'])['Address - Country'] \
                           .transform('nunique')

# keep only rows where theres > 1 Address - Country 
tab5 = tab5[tab5['unique_count'] > 1]

# remove duplicates
tab5_columns = tab5.drop_duplicates(subset=['Party ID','key','Address - Country'])

# state inconsistency type in new column 'Inconsistency'
tab5_columns['Inconsistency'] = 'Same Name, DOB, Diff Address Country'

In [ ]:
# np6: same name & dob & id# but diff id doc type
# location: sheet 1
tab6 = df_natural.copy()

# create concat column of same party name, DOB, Identification Document Number
tab6['key'] = tab6['Party Name_pre'].astype(str) + \
              tab6['Date of Birth'].astype(str) + \
              tab6['Identification Document Number_pre'].astype(str)

# add a column to count no. of Identification Document Type based on same party name, DOB & Identification Document Number
tab6['unique_count'] = tab6.groupby(['key'])['Identification Document Type'] \
                           .transform('nunique')

# keep only rows where theres > 1 Identification Document Type 
tab6 = tab6[tab6['unique_count'] > 1]

# remove duplicates
tab6_columns = tab6.drop_duplicates(subset=['Party ID','key','Identification Document Type'])

# state inconsistency type in new column 'Inconsistency'
tab6_columns['Inconsistency'] = 'Same Name, DOB, ID #, Diff ID Doc Type'

In [ ]:
# np7: same name & dob & id# but diff id expiry
# location: sheet 1
tab7 = df_natural.copy()

#Treat ID Document Expiry Date as a string
tab7['Identification Document Expiry Date_1'] = tab7['Identification Document Expiry Date'].astype(str)

# create concat column of same party name, DOB, Identification Document Number
tab7['key'] = tab7['Party Name_pre'].astype(str) + \
              tab7['Date of Birth'].astype(str) + \
              tab7['Identification Document Number_pre'].astype(str)

# add a column to count no. of Identification Document Expiry Date based on same party name, DOB & Identification Document Number
tab7['unique_count'] = tab7.groupby(['key'])['Identification Document Expiry Date_1'] \
                           .transform('nunique')

# keep only rows where theres > 1 Identification Document Expiry Date 
tab7 = tab7[tab7['unique_count'] > 1]

#groupby party name/DOB/ID and party RM (RM UID.1)
tab7['unique_count_party'] = tab7.groupby(['key'])['RM UID.1'] \
                                 .transform('nunique')

tab7_exclude = tab7[(tab7['unique_count_party'] > 1) & 
                    (tab7['Identification Document Type'] == 'Declaration of Nationality')]

#drop rows to be excluded
tab7 = tab7.drop(tab7_exclude.index)

# remove duplicates
tab7_columns = tab7.drop_duplicates(subset = ['Party ID','key','Identification Document Expiry Date_1'])

# state inconsistency type in new column 'Inconsistency'
tab7_columns['Inconsistency'] = 'Same Name, DOB, ID #, Diff ID Expiry'

In [ ]:
# np8: same name & dob & id# but diff id country
# location: sheet 1
tab8 = df_natural.copy()

# create concat column of same party name, DOB, Identification Document Number
tab8['key'] = tab6['Party Name_pre'].astype(str) + \
              tab8['Date of Birth'].astype(str) + \
              tab8['Identification Document Number_pre'].astype(str)

# add a column to count no. of Identification Document Issue Country based on same party name, DOB & Identification Document Number
tab8['unique_count'] = tab8.groupby(['key'])['Identification Document Issue Country'] \
                           .transform('nunique')

# keep only rows where theres > 1 Identification Document Issue Country 
tab8 = tab8[tab8['unique_count'] > 1]

# remove duplicates
tab8_columns = tab8.drop_duplicates(subset=['key','Identification Document Issue Country'])

# state inconsistency type in new column 'Inconsistency'
tab8_columns['Inconsistency'] = 'Same Name, DOB, ID #, Diff ID Doc Country'

In [ ]:
# np9: same name & dob & id# but diff id issue date
# location: sheet 1
tab9 = df_natural.copy()

# create concat column of same party name, DOB, Identification Document Number
tab9['key'] = tab9['Party Name_pre'].astype(str) + \
              tab9['Date of Birth'].astype(str) + \
              tab9['Identification Document Number_pre'].astype(str)

#Treat ID date of issue as a string
tab9['Identification Date of Issue_1'] = tab9['Identification Date of Issue'].astype(str)


# add a column to count no. of Identification Date of Issue based on same party name, DOB & Identification Document Number
tab9['unique_count'] = tab9.groupby(['key'])['Identification Date of Issue_1'] \
                           .transform('nunique')

# keep only rows where theres > 1 Identification Date of Issue 
tab9 = tab9[tab9['unique_count'] > 1]

#groupby party name/DOB/ID and party RM (RM UID.1)
tab9['unique_count_party'] = tab9.groupby(['key'])['RM UID.1'] \
                                 .transform('nunique')

tab9_exclude = tab9[(tab9['unique_count_party'] > 1) & 
                    (tab9['Identification Document Type'] == 'Declaration of Nationality')]

#drop rows
tab9 = tab9.drop(tab9_exclude.index)

# remove duplicates
tab9_columns = tab9.drop_duplicates(subset = ['key','Identification Date of Issue_1'])

# state inconsistency type in new column 'Inconsistency'
tab9_columns['Inconsistency'] = 'Same Name, DOB, ID #, Diff ID Issue date'

In [ ]:
# np10: same name & dob & id type & id coountry but diff id#
# location: sheet 1
tab10 = df_natural.copy()

# create concat column of same party name, DOB, Identification Document Type, Identification Document Issue Country 
tab10['key'] = tab10['Party Name_pre'].astype(str) + \
               tab10['Date of Birth'].astype(str) + \
               tab10['Identification Document Type'].astype(str) + \
               tab10['Identification Document Issue Country'].astype(str)

# add a column to count no. of Identification Document Number based on same party name, DOB, Identification Document Type,Identification Document Issue Country
tab10['unique_count'] = tab10.groupby(['key'])['Identification Document Number_pre'] \
                             .transform('nunique')

# keep only rows where theres > 1 Identification Document Number 
tab10 = tab10[tab10['unique_count'] > 1]

# remove duplicates
tab10_columns = tab10.drop_duplicates(subset = ['Party ID','key','Identification Document Number_pre'])

# state inconsistency type in new column 'Inconsistency'
tab10_columns['Inconsistency'] = 'Same Name, DOB, ID Doc Type, ID Doc Country, Diff ID'

In [ ]:
# np11: same name & dob & party rm but diff party id
# location: sheet 1
tab11 = df_natural.copy()

# create concat column of same party name, DOB, Party RM
tab11['key'] = tab11['Party Name_pre'].astype(str) + \
               tab11['Date of Birth'].astype(str) + \
               tab11['RM Name.1'].astype(str)

# add a column to count no. of Party ID based on same party name, DOB & Party RM
tab11['unique_count'] = tab11.groupby(['key'])['Party ID'] \
                             .transform('nunique')

# keep only rows where theres > 1 Party ID 
tab11 = tab11[tab11['unique_count'] > 1]

# remove duplicates
tab11_columns = tab11.drop_duplicates(subset = ['key','Party ID'])

# state inconsistency type in new column 'Inconsistency'
tab11_columns['Inconsistency'] = 'Same Name, DOB, Party RM, Diff Party ID'

In [ ]:
# np11A: aame name & same id type & id# but diff dob (Sheet 1)
# location: sheet 1
tab11a = df_natural.copy()

#Treat DOB as a string
tab11a['Date of Birth_1'] = tab11a['Date of Birth'].astype(str)

# create concat column of same party name, DOB, Party RM
tab11a['key'] = tab11a['Party Name_pre'].astype(str) + \
                tab11a['Identification Document Type'].astype(str) + \
                tab11a['Identification Document Number'].astype(str)

# add a column to count no. of Party ID based on same party name, DOB & Party RM
tab11a['unique_count'] = tab11a.groupby(['key'])['Date of Birth_1'] \
                               .transform('nunique')

# keep only rows where theres > 1 Party ID 
tab11a = tab11a[tab11a['unique_count'] > 1]

# remove duplicates
tab11a_columns = tab11a.drop_duplicates(subset = ['key','Date of Birth'])

# state inconsistency type in new column 'Inconsistency'
tab11a_columns['Inconsistency'] = 'Same Name, Same ID Doc Type, ID #, Diff DoB'

In [ ]:
# S12: Same ID Doc Type, ID #, ID Doc Country, Diff Name (Sheet 2)
tab12 = df_natural.copy()

# Check if Identification Document Number contains invalid value
tab12['invalid_identification_document_number'] = tab12['Identification Document Number'].isin(['[]','()', np.nan,'NA', 
                                                                                                'NA in iCare', 
                                                                                                'N.A In Icare', 
                                                                                                'n/a','na','N/A','N.A.',
                                                                                                'N.A','indeterminata',
                                                                                                'Indeterminata',' ','','.',
                                                                                                '-','0','0000000','000000',
                                                                                                'not available', 
                                                                                                'not applicable', 
                                                                                                'NIL', 'xx', 'not','XX', 
                                                                                                'Not available',
                                                                                                '[Not Available]',
                                                                                                '[Un Specified]'])


#Filter for records where ID Doc Type / ID# / ID Doc Country is not blank or invalid
tab12 = tab12[(tab12['Identification Document Type'] != "") & 
              (tab12['invalid_identification_document_number'] == False) &
              (tab12['Identification Document Issue Country'] != "")]

# create concat column
tab12['key'] = tab12['Identification Document Type'].astype(str) + \
               tab12['Identification Document Issue Country'].astype(str) + \
               tab12['Identification Document Number_pre'].astype(str)

# add a column to count no. of party name
tab12['unique_count'] = tab12.groupby(['key'])['Party Name_pre'] \
                             .transform('nunique')

# keep only rows where theres > 1 name
tab12 = tab12[tab12['unique_count'] > 1]

# drop duplicates
#tab12 = tab12.sort_values(by=['Party Name_pre'])
tab12_columns = tab12.drop_duplicates(subset = ['Party ID','key','Party Name_pre'])

# state inconsistency type in new column 'Inconsistency'
tab12_columns['Inconsistency'] = 'Same ID Doc Type, ID #, ID Doc Country, Diff Name'

In [ ]:
# S13: DON expiry – issue date != 2 years or missing (Sheet 3)
tab13 = df_natural.copy()

# filter for Identification Document Type == Declaration of Nationality
tab13 = tab13[tab13['Identification Document Type'] == 'Declaration of Nationality']

# calculate time delta between Identification Document Expiry Date & Identification Document Expiry Date
tab13['days_diff'] = tab13['Identification Document Expiry Date'] - tab13['Identification Date of Issue']

# filter for days_diff > 731
tab13_columns = tab13[(tab13['days_diff'] > dt.timedelta(731)) | 
                      (tab13['days_diff'].isna())]

# state inconsistency type in new column 'Inconsistency'
tab13_columns['Inconsistency'] = 'DON expiry – issue date != 2 years or missing'

In [ ]:
# S14: Invalid Place of Birth (Sheet 3)
tab14 = df_natural.copy()

# extract rows which Place of Birth contains invalid value
tab14['invalid_place_of_birth'] = tab14['Place of Birth'].isin(['[]','()', np.nan,'NA', 'NA in iCare', 'N.A In Icare', 
                                                                'n/a','na','N/A','N.A.','N.A','indeterminata',
                                                                'Indeterminata',' ','','.','-','0','0000000',
                                                                '000000','not available', 'not applicable', 
                                                                'NIL', 'xx', 'not','XX', 'Not available','[Not Available]',
                                                                '[Un Specified]'])
tab14_filter = tab14[tab14['invalid_place_of_birth'] == True]

tab14_columns = tab14_filter.drop_duplicates(subset = ['Party ID'])

tab14_columns['Inconsistency'] = 'Invalid Place of Birth'

In [ ]:
# S15: Invalid ID Doc Number (Sheet 3)
tab15 = df_natural.copy()

# extract rows which Identification Document Number contains invalid value
tab15['invalid_identification_document_number'] = tab15['Identification Document Number'].isin(['[]','()', np.nan,'NA', 
                                                                                                'NA in iCare', 
                                                                                                'N.A In Icare', 
                                                                                                'n/a','na','N/A','N.A.',
                                                                                                'N.A','indeterminata',
                                                                                                'Indeterminata',' ','','.',
                                                                                                '-','0','0000000','000000',
                                                                                                'not available', 
                                                                                                'not applicable', 
                                                                                                'NIL', 'xx', 'not','XX', 
                                                                                                'Not available',
                                                                                                '[Not Available]',
                                                                                                '[Un Specified]'])
tab15_filter = tab15[(tab15['invalid_identification_document_number'] == True) &
                     (tab15['Identification Document Type']!='Declaration of Nationality')]

tab15_columns = tab15_filter.drop_duplicates(subset = ['Party ID',
                                                       'Identification Document Type',
                                                       'Identification Document Issue Country',
                                                       'Identification Document Number'])

tab15_columns['Inconsistency'] = 'Invalid ID Doc #'

In [ ]:
# S16: ID Doc Issue Country is blank when ID doc type is not blank (Sheet 3)
tab16 = df_natural.copy()

tab16_filter = tab16[~tab16['Identification Document Type'].isna()]

tab16_filter_2 = tab16_filter[tab16_filter['Identification Document Issue Country'].isna()]

tab16_columns = tab16_filter_2.drop_duplicates(subset = ['Party ID',
                                                         'Identification Document Type',
                                                         'Identification Document Issue Country'])

# state inconsistency type in new column 'Inconsistency'
tab16_columns['Inconsistency'] = 'ID Doc Issue Country is blank when ID doc type is not blank'

In [ ]:
# S17: Party ID tagged to 2 portfolio RMs (Party Role = AH & BO only) (Sheet 3)
tab17 = df_natural.copy()

tab17_filter = tab17[(tab17['Relationship (Party Role)'] == 'Account Holder') |
                     (tab17['Relationship (Party Role)'] == 'Beneficial Owner')]

# add a column to count no. of party name
tab17_filter['unique_count'] = tab17_filter.groupby(['Party ID'])['RM UID'] \
                                           .transform('nunique')

# keep only rows where theres > 1 Identification Document Type 
tab17_filter = tab17_filter[tab17_filter['unique_count'] > 1]

#drop duplicates
tab17_columns = tab17_filter.drop_duplicates(subset = ['Party ID','RM UID'])

# state inconsistency type in new column 'Inconsistency'
tab17_columns['Inconsistency'] = 'Party ID tagged to 2 portfolio RMs (Party Role = AH & BO only)'

In [ ]:
# S18: Party ID tagged to multiple Party RMs (Sheet 3)
tab18 = df_natural.copy()

tab18['unique_count'] = tab18.groupby(['Party ID'])['RM UID.1'] \
                             .transform('nunique')

tab18 = tab18[tab18['unique_count'] > 1]

tab18 = tab18.sort_values(by = ['Party Name_pre'])

tab18_columns = tab18.drop_duplicates(subset = ['Party ID','RM UID.1'])

# state inconsistency type in new column 'Inconsistency'
tab18_columns['Inconsistency'] = 'Party ID tagged to multiple Party RMs'

In [ ]:
# S21: ID only and not from acceptable countries (Sheet 1)
tab21 = df_natural.copy()

# create group identifier: [key]
tab21['key'] = tab21['Party Name_pre'].astype(str) + tab21['Date of Birth'].astype(str)

# count of ID doc type based on same name and DOB
tab21['unique_count'] = tab21.groupby(['key'])['Identification Document Type'] \
                             .transform('nunique')

# keep only rows where theres only 1 ID doc type and the ID doc type is ID/NRIC
tab21_1 = tab21[(tab21['unique_count'] == 1) & (tab21['Identification Document Type'] == 'SG Pink NRIC')]
tab21_2 = tab21[(tab21['unique_count'] == 1) & (tab21['Identification Document Type'] == 'National ID')]

# Check if Nationality = Singaporean
tab21_1_error = tab21_1[tab21_1['Nationality'] != 'Singapore']

# Check if Nationality = acceptable countries
country_list = ['Switzerland', 'Italy', 'Germany', 'France', 'Princip.Liechtenstein', 'Thailand', 'Singapore']
tab21_2_error = tab21_2[~tab21_2['Nationality'].isin(country_list)]

# v-lookup in originial df 
tab21_1_error_list = tab21_1_error['key'].tolist()
tab21_2_error_list = tab21_2_error['key'].tolist()
tab21_error_list = tab21_1_error_list + tab21_2_error_list

tab21['exist'] = tab21['key'].isin(tab21_error_list)
tab21_columns = tab21[tab21['exist'] == True]

# state inconsistency type in new column 'Inconsistency'
tab21_columns['Inconsistency'] = 'ID only and not from acceptable countries'

#drop duplicates
tab21_columns = tab21_columns.drop_duplicates(subset = ['Party ID', 'Nationality'])

In [ ]:
# S22: Blue NRIC not SG Resident (Sheet 1)
tab22 = df_natural.copy()

# create group identifier: [key]
tab22['key'] = tab22['Party Name_pre'].astype(str) + tab22['Date of Birth'].astype(str)

# count of ID doc type based on same name and DOB
tab22['unique_count'] = tab22.groupby(['key'])['Identification Document Type'] \
                             .transform('nunique')

# keep only rows where theres only 1 ID doc type and the ID doc type is Blue NRIC
tab22 = tab22[(tab22['unique_count'] == 1) & 
              (tab22['Identification Document Type'] == 'SG Blue NRIC')]

# Residential Pass Holder of = Singapore
# Address type = Existing Residential Address 
# Address - Country = Singapore
tab22_columns = tab22[~((tab22["Resident pass holder of"] == 'Singapore') &
                        (tab22["Address Type"] == 'Existing Residential Address') &
                        (tab22["Address - Country"] == 'Singapore')
                       )
                     ]

# state inconsistency type in new column 'Inconsistency'
tab22_columns['Inconsistency'] = 'Blue NRIC not SG Resident'

#drop duplicates
tab22_columns = tab22_columns.drop_duplicates(subset = ['Party ID', 'Nationality'])

In [ ]:
# S23: Invalid override expiry reason (Sheet 5)
tab23 = df_natural.copy()

#create group identifier: [key]
tab23['key']=tab23['Party Name_pre'].astype(str) + tab23['Date of Birth'].astype(str)

doc_types = ['SG Pink NRIC', 'National ID', 'Passport']
tab23 = tab23[tab23["Identification Document Type"].isin(doc_types)]

tab23 = tab23[(tab23['Override Expiry'] == 'Yes') & 
              (tab23['Override Expiry Reason'] != 'Deceased')]

tab23_columns = tab23.drop_duplicates(subset = ['Party ID'])

# state inconsistency type in new column 'Inconsistency'
tab23_columns['Inconsistency'] = 'Invalid override expiry reason'

In [ ]:
# WIP S24: Override and deceased flag mismatch
#S24: override reason and deceased flag mismatch
#override=yes AND reason=deceased AND flag!=deceased
#override=yes AND reason!=deceased AND flag=deceased
#(override=no AND reason!=blank) OR (override=no AND flag=deceased)
#try to combine if possible
tab24 = df_natural.copy()

#create group identifier: [key]
tab24['key']=tab24['Party Name_pre'].astype(str) + tab24['Date of Birth'].astype(str)

doc_types = ['SG Pink NRIC', 'National ID', 'Passport']
tab24 = tab24[tab24["Identification Document Type"].isin(doc_types)]

tab24 = tab24[(cond1) |
              (cond2)]

tab24_columns = tab24.drop_duplicates(subset = ['Party ID'])

# state inconsistency type in new column 'Inconsistency'
tab24_columns['Inconsistency'] = 'Override and Deceased Flag Mismatch'

***

In [ ]:
# sheet 1
df_1 = [tab1_columns,tab2_columns,tab4_columns,tab5_columns,tab6_columns,tab7_columns,tab8_columns,tab9_columns,
        tab10_columns,tab11_columns,tab11a_columns,tab21_columns, tab22_columns]

result_1 = pd.concat(df_1, join='outer', axis=0)
result_1 = result_1.sort_values(by=['Party Name_pre','Inconsistency','Party ID'])
result_1['Beneficiary_role_only']=result_1['Party ID'].isin(bene_only_list)

# select columns
result_1 = result_1[['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date',
                    'Party ID','Party Name','RM UID.1','RM Name.1','Date of Birth','Place of Birth','Resident pass holder of',
                    'Address Type','Address - Country','Nationality','Identification Document Type','Identification Document Issue Country',
                    'Identification Document Number','Identification Date of Issue','Identification Document Expiry Date',
                    'Beneficiary_role_only','HK Permanent ID?']]

#rename columns
result_1.columns = ['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date',
                    'Party ID','Party Name','Party RM UID','Party RM Name','Date of Birth','Place of Birth','Resident pass holder of',
                    'Address Type','Address - Country','Nationality','Identification Document Type','Identification Document Issue Country',
                    'Identification Document Number','Identification Date of Issue','Identification Document Expiry Date',
                    'Beneficiary_role_only','HK Permanent ID?']

In [ ]:
# sheet 2
result_2 = tab12_columns.sort_values(by=['key','Party Name_pre','Party ID'])
result_2['Beneficiary_role_only']=result_2['Party ID'].isin(bene_only_list)

# select columns
result_2 = result_2[['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date',
                    'Party ID','Party Name','RM UID.1','RM Name.1','Date of Birth',
                    'Identification Document Type','Identification Document Issue Country','Identification Document Number',
                    'Beneficiary_role_only']]
# rename columns
result_2.columns = ['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date',
                    'Party ID','Party Name','Party RM UID','Party RM Name','Date of Birth',
                    'Identification Document Type','Identification Document Issue Country','Identification Document Number',
                    'Beneficiary_role_only']

In [ ]:
# sheet 3
df_3 = [tab13_columns,tab14_columns,tab15_columns,tab16_columns,tab17_columns,tab18_columns]

result_3 = pd.concat(df_3, join='outer', axis=0)
result_3 = result_3.sort_values(by=['Party Name_pre','Inconsistency','Party ID'])
result_3['Beneficiary_role_only']=result_3['Party ID'].isin(bene_only_list)

# select columns
result_3 = result_3 [['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date',
                    'Party ID','Party Name','RM UID.1','RM Name.1','RM UID','RM Name','Date of Birth','Place of Birth',
                    'Identification Document Type','Identification Document Issue Country','Identification Document Number',
                    'Identification Date of Issue','Identification Document Expiry Date','Beneficiary_role_only']]

# rename columns
result_3.columns = ['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date',
                    'Party ID','Party Name','Party RM UID','Party RM Name','Portfolio RM UID','Portfolio RM Name','Date of Birth','Place of Birth',
                    'Identification Document Type','Identification Document Issue Country','Identification Document Number',
                    'Identification Date of Issue','Identification Document Expiry Date','Beneficiary_role_only']

In [ ]:
# sheet 5
df_5 = [tab23_columns,tab24_columns]

result_5 = df_5
result_5 = result_5.sort_values(by=['Party Name_pre','Inconsistency','Party ID'])
result_5['Beneficiary_role_only']=result_5['Party ID'].isin(bene_only_list)

# select columns
result_5 = result_5[['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date',
                    'Party ID', 'Party Name','RM UID.1','RM Name.1','Date of Birth',
                    'Identification Document Type','Identification Document Number','Identification Document Issue Country',
                    'Identification Date of Issue','Identification Document Expiry Date',
                    'Override Expiry','Override Expiry Reason','Override Expiry Justification',
                    'Deceased Flag','Beneficiary_role_only']]

# rename columns
result_5.columns = ['Batch','Inconsistency','Remarks','Action Required','Action Team','Status of Cleanup','Completion Date',
                    'Party ID', 'Party Name','Party RM UID','Party RM Name','Date of Birth',
                    'Identification Document Type','Identification Document Number','Identification Document Issue Country',
                    'Identification Date of Issue','Identification Document Expiry Date',
                    'Override Expiry','Override Expiry Reason','Override Expiry Justification',
                    'Deceased Flag','Beneficiary_role_only']

In [ ]:
writer = ExcelWriter(folder_to_save_files, mode='w',date_format = 'yyyy-mm-dd', 
                        datetime_format='yyyy-mm-dd')

result_1.to_excel(writer, 'Same_name_diff_static_info', index=False) #tab1
result_2.to_excel(writer, 'Same_ID_diff_name', index=False) #tab2
result_3.to_excel(writer, 'incomplete_fields_or_2RM', index=False) #tab3
result_5.to_excel(writer, 'Invalid_override_expiry', index=False) #tab5

writer.save()

In [ ]:
print("Completed.")